In [1]:
import os,sys

notebook_dir = os.getcwd()
print('notebook dir: ', notebook_dir)
project_dir = os.path.dirname(notebook_dir)
print('project dir: ', project_dir)
sys.path.insert(0, project_dir)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "dingdong.settings")

import django
django.setup()


def print_qs(querySet):
    from django.core import serializers
    data = serializers.serialize("json", querySet, indent=4)
    print(data)

notebook dir:  D:\workspace\python\dingdong\notebooks
project dir:  D:\workspace\python\dingdong


In [2]:
from base.models import Account
from django.contrib.auth.models import Group
from django.contrib.auth.models import Permission
from django.contrib.auth import authenticate
from django.contrib.contenttypes.models import ContentType

In [9]:
from rest_framework import serializers
from rest_framework.renderers import JSONRenderer
from base.api.serializers import AccountSerializer
from base.api.serializers import GroupSerializer

In [15]:
class AccountSerializer(serializers.ModelSerializer):
    groups = GroupSerializer(many=True)
    class Meta:
        model = Account
        fields = ('username', 'email', 'groups')
serializer = AccountSerializer()
print(repr(serializer))

AccountSerializer():
    username = CharField(help_text='Required. 150 characters or fewer. Letters, digits and @/./+/-/_ only.', max_length=150, validators=[<django.contrib.auth.validators.UnicodeUsernameValidator object>, <UniqueValidator(queryset=Account.objects.all())>])
    email = EmailField(allow_blank=True, label='Email address', max_length=254, required=False)
    groups = GroupSerializer(many=True):
        id = IntegerField(label='ID', read_only=True)
        name = CharField(max_length=80, validators=[<UniqueValidator(queryset=Group.objects.all())>])
        permissions = PrimaryKeyRelatedField(many=True, queryset=Permission.objects.all(), required=False)


In [16]:
serializer = AccountSerializer(Account.objects.all(), many=True)
serializer.data

[OrderedDict([('username', 'admin'), ('email', 'odingdongo@hotmail.com'), ('groups', [])]), OrderedDict([('username', 'ada'), ('email', 'ada@glittergroupcn.com'), ('groups', [OrderedDict([('id', 1), ('name', 'salesperson'), ('permissions', [52, 60])]), OrderedDict([('id', 3), ('name', 'inventory'), ('permissions', [31, 35])])])])]

In [14]:
class GroupSerializer(serializers.ModelSerializer):

    class Meta:
        model = Group
        fields = '__all__'

In [ ]:
class ContentTypeSerializer(serializers.ModelSerializer):
    class Meta:
        model = ContentType
        # fields = ('codename', 'content_type', 'name')
        fields = '__all__'

In [ ]:
serializer = ContentTypeSerializer(ContentType.objects.all(), many=True)

In [25]:
class PermissionSerializer(serializers.ModelSerializer):
    # content_type = ContentTypeSerializer()
    app_label = serializers.CharField(source='content_type.app_label', read_only=True)
    model = serializers.CharField(source='content_type.model', read_only=True)
    content_type_id = serializers.IntegerField(source='content_type.id', read_only=True)
    class Meta:
        model = Permission
        fields = ('codename', 'app_label', 'model', 'content_type_id', 'name')
        # fields = '__all__'

In [26]:
serializer = PermissionSerializer()
print(repr(serializer))

PermissionSerializer():
    codename = CharField(max_length=100)
    app_label = CharField(read_only=True, source='content_type.app_label')
    model = CharField(read_only=True, source='content_type.model')
    content_type_id = IntegerField(read_only=True, source='content_type.id')
    name = CharField(max_length=255)


In [27]:
serializer = PermissionSerializer(ada.user_permissions.all(), many=True)

In [28]:
JSONRenderer().render(serializer.data)
serializer.data

[OrderedDict([('codename', 'add_article'), ('app_label', 'base'), ('model', 'article'), ('content_type_id', 8), ('name', 'Can add article')]), OrderedDict([('codename', 'add_articlecategory'), ('app_label', 'base'), ('model', 'articlecategory'), ('content_type_id', 9), ('name', 'Can add article category')]), OrderedDict([('codename', 'delete_company'), ('app_label', 'base'), ('model', 'company'), ('content_type_id', 10), ('name', 'Can delete company')]), OrderedDict([('codename', 'change_customer'), ('app_label', 'base'), ('model', 'customer'), ('content_type_id', 13), ('name', 'Can change customer')]), OrderedDict([('codename', 'add_staff'), ('app_label', 'base'), ('model', 'staff'), ('content_type_id', 15), ('name', 'Can add 员工')])]

In [22]:
ada = Account.objects.get(username='ada')

In [21]:
print_qs(ada.user_permissions.all())

[
{
    "model": "auth.permission",
    "pk": 29,
    "fields": {
        "name": "Can add article",
        "content_type": 8,
        "codename": "add_article"
    }
},
{
    "model": "auth.permission",
    "pk": 33,
    "fields": {
        "name": "Can add article category",
        "content_type": 9,
        "codename": "add_articlecategory"
    }
},
{
    "model": "auth.permission",
    "pk": 39,
    "fields": {
        "name": "Can delete company",
        "content_type": 10,
        "codename": "delete_company"
    }
},
{
    "model": "auth.permission",
    "pk": 50,
    "fields": {
        "name": "Can change customer",
        "content_type": 13,
        "codename": "change_customer"
    }
},
{
    "model": "auth.permission",
    "pk": 57,
    "fields": {
        "name": "Can add \u5458\u5de5",
        "content_type": 15,
        "codename": "add_staff"
    }
}
]

